In [1]:
import os, shutil, random, glob
import cv2
import numpy as np
import pandas as pd

# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# CUDA_VISIBLE_DEVICES = 2

import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
resize = 150
def load_data():
    imgs = os.listdir("./train/")
    num = len(imgs)
    data = np.empty((3000, resize, resize, 3), dtype="int32")
    label = np.empty((3000, ), dtype="int32")
    for i in range(3000):
        if i%2:
            data[i] = cv2.resize(cv2.imread('./train/' + 'dog.' + str(i) + '.jpg'), (resize, resize))
            label[i] = 1
        else:
            data[i] = cv2.resize(cv2.imread('./train/' + 'cat.' + str(i) + '.jpg'), (resize, resize))
            label[i] = 0
    return data,label

In [3]:
data,label = load_data()
data = data.astype('float32')
data /= 255

In [4]:
# y_train = keras.utils.to_categorical(y_train, 2)
# y_val = keras.utils.to_categorical(y_val, 2)
# label = keras.utils.to_categorical(label, 2)

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (resize, resize, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [10]:
model.fit(data, label,
          batch_size = 128,
          epochs = 30,
          validation_split = 0.2,
          shuffle = True)

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 2s 907us/step - loss: 0.0856 - acc: 0.9821 - val_loss: 1.5912 - val_acc: 0.7900
Epoch 2/30
2400/2400 [==============================] - 2s 637us/step - loss: 0.1474 - acc: 0.9646 - val_loss: 1.2177 - val_acc: 0.7750
Epoch 3/30
2400/2400 [==============================] - 2s 639us/step - loss: 0.0467 - acc: 0.9883 - val_loss: 1.1409 - val_acc: 0.7850
Epoch 4/30
2400/2400 [==============================] - 2s 627us/step - loss: 0.0478 - acc: 0.9900 - val_loss: 1.2283 - val_acc: 0.6900
Epoch 5/30
2400/2400 [==============================] - 2s 674us/step - loss: 0.0969 - acc: 0.9700 - val_loss: 1.4023 - val_acc: 0.7733
Epoch 6/30
2400/2400 [==============================] - 2s 629us/step - loss: 0.0568 - acc: 0.9875 - val_loss: 2.2483 - val_acc: 0.7650
Epoch 7/30
2400/2400 [==============================] - 2s 676us/step - loss: 0.1058 - acc: 0.9679 - val_loss: 1.3947 - val_acc: 0.7750
E

In [7]:
test_length = len(os.listdir('./test/'))
test_data = np.empty((test_length, resize, resize, 3), dtype="int32")

test_data = test_data.astype('float32')
test_data /= 255

##生成test image
for file_name in os.listdir('./test/'):
    index = int(file_name.split('.')[0]) - 1  #文件名和默认的index相差1
    test_data[index] = cv2.resize(cv2.imread('./test/' + file_name), (resize, resize))

In [8]:
test_label = model.predict(test_data)
test_label = test_label.astype('int32')
test_label = test_label.reshape(-1)
index = np.arange(12500) + 1

result = pd.DataFrame({'id': index, 'label': test_label})
result.to_csv('result.csv', index = False)